# RAG from Oracle #

In [1]:
%%capture --no-stderr
%pip install -qU  langchain langchain-community langchain-openai langgraph oracledb


In [2]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass()

# Comment out the below to opt-out of using LangSmith in this notebook. Not required.
if not os.environ.get("LANGCHAIN_API_KEY"):
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
    os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [3]:

from langchain_community.utilities import SQLDatabase
import oracledb

db_uri = "oracle+oracledb://hr:hr@localhost:1521?service_name=xepdb1"
db = SQLDatabase.from_uri(db_uri)

print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM departments")



oracle
['countries', 'departments', 'emp_temp', 'employees', 'j_purchaseorder', 'job_history', 'jobs', 'locations', 'regions', 'sales']


"[(10, 'Administration', 200, 1700), (20, 'Marketing', 201, 1800), (30, 'Purchasing', 114, 1700), (40, 'Human Resources', 203, 2400), (50, 'Shipping', 121, 1500), (60, 'IT', 103, 1400), (70, 'Public Relations', 204, 2700), (80, 'Sales', 145, 2500), (90, 'Executive', 100, 1700), (100, 'Finance', 108, 1700), (110, 'Accounting', 205, 1700), (120, 'Treasury', None, 1700), (130, 'Corporate Tax', None, 1700), (140, 'Control And Credit', None, 1700), (150, 'Shareholder Services', None, 1700), (160, 'Benefits', None, 1700), (170, 'Manufacturing', None, 1700), (180, 'Construction', None, 1700), (190, 'Contracting', None, 1700), (200, 'Operations', None, 1700), (210, 'IT Support', None, 1700), (220, 'NOC', None, 1700), (230, 'IT Helpdesk', None, 1700), (240, 'Government Sales', None, 1700), (250, 'Retail Sales', None, 1700), (260, 'Recruiting', None, 1700), (270, 'Payroll', None, 1700)]"

In [4]:
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits import SQLDatabaseToolkit

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
#llm = ChatOpenAI(
#    model="gpt-4o",
#    temperature=0,
#    max_tokens=None,
#    timeout=None,
#    max_retries=2,
#)

toolkit = SQLDatabaseToolkit(db=db, llm=llm)
tools = toolkit.get_tools()
tools

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x110b8d580>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x110b8d580>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x110b8d580>),
 QuerySQLCheckerTool(description='Use this tool to double check if your 

In [5]:
from langchain_core.messages import SystemMessage

SQL_PREFIX = """You are an agent designed to interact with an Oracle SQL database.
Given an input question, create a syntactically correct Oracle SQL query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP, TRUNCATE etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query. 
Do NOT skip this step.
When possible try to join tables to extend the retrieved information.
Then you should query the schema of the most relevant tables."""

system_message = SystemMessage(content=SQL_PREFIX)
system_message.pretty_print

<bound method BaseMessage.pretty_print of SystemMessage(content='You are an agent designed to interact with an Oracle SQL database.\nGiven an input question, create a syntactically correct Oracle SQL query to run, then look at the results of the query and return the answer.\nUnless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.\nYou can order the results by a relevant column to return the most interesting examples in the database.\nNever query for all the columns from a specific table, only ask for the relevant columns given the question.\nYou have access to tools for interacting with the database.\nOnly use the below tools. Only use the information returned by the below tools to construct your final answer.\nYou MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.\n\nDO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP, TRUNCATE etc.) 

In [6]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools, messages_modifier=system_message)



In [ ]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content="Which are the three departments with the most employees?. Show only the department's name")]}
):
    print(s)
    print("----")

In [7]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content="what's the salary of John?. List all of them including the full name and salary")]}
):
    print(s)

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_KrZWvHaPqIyCNmwy7vEsZw9U', 'function': {'arguments': '{}', 'name': 'sql_db_list_tables'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 585, 'total_tokens': 597}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6f5aee92-ad74-47b3-88e4-8773002e93ec-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {}, 'id': 'call_KrZWvHaPqIyCNmwy7vEsZw9U'}], usage_metadata={'input_tokens': 585, 'output_tokens': 12, 'total_tokens': 597})]}}
{'tools': {'messages': [ToolMessage(content='countries, departments, emp_temp, employees, j_purchaseorder, job_history, jobs, locations, regions, sales', name='sql_db_list_tables', tool_call_id='call_KrZWvHaPqIyCNmwy7vEsZw9U')]}}
{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_yyswONi9kEuPE0dVrxKzXuWS'

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_community/utilities/sql_database.py:317: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "departments, employees", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_community/utilities/sql_database.py:329: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "departments, employees", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables


{'tools': {'messages': [ToolMessage(content='\nCREATE TABLE employees (\n\temployee_id NUMBER(6, 0) NOT NULL, \n\tfirst_name VARCHAR(20 CHAR), \n\tlast_name VARCHAR(25 CHAR) NOT NULL, \n\temail VARCHAR(25 CHAR) NOT NULL, \n\tphone_number VARCHAR(20 CHAR), \n\thire_date DATE NOT NULL, \n\tjob_id VARCHAR(10 CHAR) NOT NULL, \n\tsalary NUMBER(8, 2), \n\tcommission_pct NUMBER(2, 2), \n\tmanager_id NUMBER(6, 0), \n\tdepartment_id NUMBER(4, 0), \n\tCONSTRAINT emp_emp_id_pk PRIMARY KEY (employee_id), \n\tCONSTRAINT emp_dept_fk FOREIGN KEY(department_id) REFERENCES departments (department_id), \n\tCONSTRAINT emp_job_fk FOREIGN KEY(job_id) REFERENCES jobs (job_id), \n\tCONSTRAINT emp_manager_fk FOREIGN KEY(manager_id) REFERENCES employees (employee_id), \n\tCONSTRAINT emp_salary_min CHECK (salary > 0)\n)\n\n/*\n3 rows from employees table:\nemployee_id\tfirst_name\tlast_name\temail\tphone_number\thire_date\tjob_id\tsalary\tcommission_pct\tmanager_id\tdepartment_id\n199\tDouglas\tGrant\tDGRANT\t1

In [8]:
def show_final_message(response):
    """Print the last message from the response"""
    message = response["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()


In [9]:
show_final_message(agent_executor.invoke(
    {"messages": [HumanMessage(content="What's John Chen's salary?")]}
    )
)



================================== Ai Message ==================================

John Chen's salary is $8,200.


In [ ]:
show_final_message( agent_executor.invoke(
    {"messages": [HumanMessage(content="What's John Chen's job title?. Give me the title name, not the title id")]}
    )
)


In [ ]:
show_final_message( agent_executor.invoke(
    {"messages": [HumanMessage(content="What's the highest salary for accountants?.")]}
    )
)

show_final_message( agent_executor.invoke(
    {"messages": [HumanMessage(content="What's the salary and position of Daniel?.")]}
    )
)


In [ ]:
show_final_message( agent_executor.invoke(
    {"messages": [HumanMessage(content="Hi, I'm Ismael Sciarra. Lookup my salary in the employee's table")]}
    )
)

show_final_message( agent_executor.invoke(
    {"messages": [HumanMessage(content="Who's my manager?.")]}
    )
)


## Adding Memory ##

In [10]:
from langgraph.checkpoint import MemorySaver
agent_memory = create_react_agent(llm, tools, checkpointer=MemorySaver())
config = {"configurable": {"thread_id": "thread-1"}}

def print_stream(graph, inputs, config):
    for s in agent_memory.stream(inputs, config, stream_mode="values"):
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

inputs = {"messages": [("user", "Hi, I'm Ismael Sciarra. Lookup my salary in the employee's table")]}
print_stream(agent_memory, inputs, config)


================================ Human Message =================================

Hi, I'm Ismael Sciarra. Lookup my salary in the employee's table
================================== Ai Message ==================================
Tool Calls:
  sql_db_query (call_W4QkodRyB3DOd4gyLBUuiUo4)
 Call ID: call_W4QkodRyB3DOd4gyLBUuiUo4
  Args:
    query: SELECT salary FROM employees WHERE first_name = 'Ismael' AND last_name = 'Sciarra'
================================= Tool Message =================================
Name: sql_db_query

[(Decimal('7700'),)]
================================== Ai Message ==================================

Ismael Sciarra's salary is $7700.


In [ ]:
config = {"configurable": {"thread_id": "thread-2"}} # new thread

print_stream(agent_memory,  {"messages": [("user", "Who's my manager?")]}, config)

In [ ]:
config = {"configurable": {"thread_id": "thread-1"}} # back to the thread-1

print_stream(agent_memory,  {"messages": [("user", "Who's my manager?")]}, config)

In [ ]:
print_stream(agent_memory,  {"messages": [("user", "Remember these three words: horse, moonlight, wizard")]}, config)

In [ ]:
print_stream(agent_memory,  {"messages": [("user", "Give me a sentence using those three words, my name and my salary")]}, config)

In [ ]:
print_stream(agent_memory,  {"messages": [("user", "find any employee making more money than me?")]}, config)


In [ ]:
config = {"configurable": {"thread_id": "thread-3"}}
print_stream(agent_memory,  {"messages": [("user", "Who is the manager of Scyara?")]}, config)



## Dealing with high-cardinality columns ##

In [ ]:
import ast
import re


def query_as_list(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
    return list(set(res))


employees = query_as_list(db, "SELECT first_name || ' ' || last_name employee_name FROM Employees")
employees[:5]

In [ ]:
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

vector_db = FAISS.from_texts(employees, OpenAIEmbeddings())
retriever = vector_db.as_retriever(search_kwargs={"k": 5})
description = """Use to look up values to filter on. Input is an approximate spelling of the proper noun, output is \
valid proper nouns. Use the noun most similar to the search."""
retriever_tool = create_retriever_tool(
    retriever,
    name="search_proper_nouns",
    description=description,
)

In [ ]:
print(retriever_tool.invoke("Scyara"))

In [ ]:
system = """You are an agent designed to interact with an Oracle SQL database.
Given an input question, create a syntactically correct Oracle SQL query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP, TRUNCATE etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query. 
Do NOT skip this step.
When possible try to join tables to extend the retrieved information.
Then you should query the schema of the most relevant tables.

If you need to filter on a proper noun, you must ALWAYS first look up the filter value using the "search_proper_nouns" tool!
Do not try to guess at the proper name - use this function to find similar ones.
"""

system_message = SystemMessage(content=system)
tools.append(retriever_tool)
new_agent = create_react_agent(llm, tools, messages_modifier=system_message, checkpointer=MemorySaver())

In [ ]:
config = {"configurable": {"thread_id": "thread-2"}}

for s in new_agent.stream(
    {"messages": [HumanMessage(content="Who is the manager of Scyara?")]}, config):
    print(s)
    print("----")

### Anonymizing ###
https://python.langchain.com/v0.1/docs/guides/productionization/safety/presidio_data_anonymization/reversible/

In [11]:
# Install necessary packages
%pip install --upgrade --quiet  langchain langchain-experimental langchain-openai presidio-analyzer presidio-anonymizer spacy Faker
# ! python -m spacy download en_core_web_lg


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
from langchain_experimental.data_anonymizer import PresidioReversibleAnonymizer

anonymizer = PresidioReversibleAnonymizer(
    analyzed_fields=["PERSON", "PHONE_NUMBER", "EMAIL_ADDRESS"],
)

from langgraph.checkpoint import MemorySaver
agent_memory = create_react_agent(llm, tools, checkpointer=MemorySaver())
config = {"configurable": {"thread_id": "anonym-thread"}}

def print_stream(graph, inputs, config):
    for s in agent_memory.stream(inputs, config, stream_mode="values"):
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

inputs = {"messages": [("user", anonymizer.anonymize("Hi, I'm Ismael Sciarra. Lookup my salary in the employee's table"))]}
print(inputs)
print_stream(agent_memory, inputs, config)


{'messages': [('user', "Hi, I'm Timothy Hicks. Lookup my salary in the employee's table")]}
================================ Human Message =================================

Hi, I'm Timothy Hicks. Lookup my salary in the employee's table
================================== Ai Message ==================================
Tool Calls:
  sql_db_query (call_jcg6vSobiaSkCYtIHGoM3l70)
 Call ID: call_jcg6vSobiaSkCYtIHGoM3l70
  Args:
    query: SELECT salary FROM employees WHERE first_name = 'Timothy' AND last_name = 'Hicks'
================================= Tool Message =================================
Name: sql_db_query


================================== Ai Message ==================================

I'm sorry, but I couldn't find any information on your salary in the database. It's possible that there is no record for 'Timothy Hicks' in the employees table. Please double-check the name or contact the database administrator for further assistance.
